Status_Calc()
    - 將股價比SP500表現好多少%變成參數, 可藉此改變outperform/underperform定義
Build_Data_Set()
    - 每檔股票過去歷史股價與SP500比較下表現: key_stats_acc_perf_WITH_NA_enhanced.csv    
Analysis()
    - 使用上述func.得到的資料當成 training data. 並以這些股票目前的股價(forward_sample_WITH_NA.csv)當成 predict data. 得到哪些股票值得投資.    

- 針對上述股票進行多次建模與預測, 若被預測可投資次數超過總預測次數1/3, 則將此股票列為可投資標的.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, preprocessing
import pandas as pd
from matplotlib import style
import statistics

from collections import Counter

style.use("ggplot")

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
how_much_better = 5

In [3]:
FEATURES =  ['DE Ratio',
             'Trailing P/E',
             'Price/Sales',
             'Price/Book',
             'Profit Margin',
             'Operating Margin',
             'Return on Assets',
             'Return on Equity',
             'Revenue Per Share',
             'Market Cap',
             'Enterprise Value',
             'Forward P/E',
             'PEG Ratio',
             'Enterprise Value/Revenue',
             'Enterprise Value/EBITDA',
             'Revenue',
             'Gross Profit',
             'EBITDA',
             'Net Income Avl to Common ',
             'Diluted EPS',
             'Earnings Growth',
             'Revenue Growth',
             'Total Cash',
             'Total Cash Per Share',
             'Total Debt',
             'Current Ratio',
             'Book Value Per Share',
             'Cash Flow',
             'Beta',
             'Held by Insiders',
             'Held by Institutions',
             'Shares Short (as of',
             'Short Ratio',
             'Short % of Float',
             'Shares Short (prior ']

In [4]:
def Status_Calc(stock, sp500):
    difference = stock - sp500

    if difference > how_much_better:
        return 1
    else:
        return 0

In [5]:
def Build_Data_Set():
    data_df = pd.DataFrame.from_csv("key_stats_acc_perf_WITH_NA_enhanced.csv")

    #data_df = data_df[:100]
    data_df = data_df.reindex(np.random.permutation(data_df.index))
    data_df = data_df.replace("NaN",0).replace("N/A",0)

    data_df["Status2"] = list(map(Status_Calc, data_df["stock_p_change"], data_df["sp500_p_change"]))
    

    X = np.array(data_df[FEATURES].values)#.tolist())

    y = (data_df["Status2"]
         .replace("underperform",0)
         .replace("outperform",1)
         .values.tolist())

    X = preprocessing.scale(X)

    Z = np.array(data_df[["stock_p_change","sp500_p_change"]])


    return X,y,Z

In [6]:
def Analysis():

    test_size = 1

    invest_amount = 10000
    
    total_invests = 0
    
    if_market = 0
    if_strat = 0
    
    X, y, Z = Build_Data_Set()
    print(len(X))

    
    clf = svm.SVC(kernel="linear", C= 1.0)
    clf.fit(X[:-test_size],y[:-test_size])

    correct_count = 0

    for x in range(1, test_size+1):
        if clf.predict(X[-x])[0] == y[-x]:
            correct_count += 1

        if clf.predict(X[-x])[0] == 1:
            invest_return = invest_amount + (invest_amount * (Z[-x][0]/100))
            market_return = invest_amount + (invest_amount * (Z[-x][1]/100))
            total_invests += 1
            if_market += market_return
            if_strat += invest_return
            


    data_df = pd.DataFrame.from_csv("forward_sample_WITH_NA.csv")

    data_df = data_df.replace("N/A",0).replace("NaN",0)

    X = np.array(data_df[FEATURES].values)

    X = preprocessing.scale(X)

    Z = data_df["Ticker"].values.tolist()

    invest_list = []

    re = clf.predict(X[:])
    for i in range(len(X)):
        if re[i] == 1:
            invest_list.append(Z[i])
            

    #print(len(invest_list))
    #print(invest_list)
    return invest_list

In [7]:
final_list = []

loops = 8

for x in range(loops):
    stock_list = Analysis()
    for e in stock_list:
        final_list.append(e)


/usr/local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-package

336
336
336
336
336
336
336
336


/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

In [8]:
x = Counter(final_list)

In [10]:
for each in x:
    if x[each] > loops - (loops/3):
        print(each)

all
gt
gs
scg
gd
kmi
lll
rop
gm
wmt
wmb
mpc
amgn
tmk
cmi
cmg
bxp
cma
cms
ups
dis
tel
cost
ter
htz
nem
abt
mchp
xom
tsla
leg
vale
met
wdc
cern
k
pbct
dri
sjm
altr
txn
win
esrx
apc
apa
apd
usb
dnb
nke
srcl
ibm
f
fls
flr
cat
cah
ms
v
cag
tap
spg
afl
hog
hon
avb
mas
mar
sna
sfly
klac
fl
a
vfc
flws
so
fslr
vmc
lsi
itw
tsn
lm
gtn
ntrs
vtr
clx
pcg
csx
goog
aep
dal
oke
tjx
nov
oxy
mnst
l
iff
dlph
el
ed
ea
sne
fisv
ew
mdt
dsw
wgo
bll
rf
aon
etfc
ecl
lmt
mmm
bwa
gis
mmc
bf-b
pbi
ati
jwn
trv
fe
xl
pgr
dgx
eqr
dfs
gps
akam
hum
rsg
pnw
shld
ben
expr
dltr
sti
dte
stt
stx
dtv
stz
anf
pru
rok
biib
xrx
cci
ctsh
pnr
bsx
vno
rht
rhi
whr
xlnx
axp
exc
camp
m
kr
atvi
mrk
dov
wynn
rtn
ctas
pki
trow
ksu
kss
hrs
hrl
hrb
ges
do
dg
mgm
disca
aes
dlx
dps
bax
orcl
aee
emr
aeo
crm
btu
nus
see
jnj
lnc
luv
mur
irm
pets
luk
omc
wtw
wm
len
pdco
wy
etr
c
dks
hban
gww
expe
cof
unh
ctxs
unm
ci
key
tgt
cb
ca
cf
znga
swk
duk
ntap
swn
px
pg
bby
ph
bbt
grmn
mck
bms
bmy
gnw
cvx
aiz
eix
aiv
cvs
tss
slm
jpm
avy
ip
ir
vz
ivz
cinf